In [52]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('../../smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.base.model import Model
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
from IPython.display import display, Markdown

#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V


3.0.0-beta


In [53]:
BSMlist = getParticlesFromSLHA('TRV1bb_slha/run_01_MZp_352_gB_4.599776E-01.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[y1]


In [54]:
slhaFolder = 'TRV1bb_slha'
resultsFolder = 'TRV1bb_results'
File = "parameters.ini"
!chmod -R +rwx {slhaFolder}

In [55]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadBinaryFile() in 542: loading binary db file /home/yoxara/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 549: Loaded database from /home/yoxara/smodels-database/db3.pcl in 2.2 secs.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 238 files with a single process. Messages will be redirected to smodels.log


In [56]:
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    #dataF = dataF.replace('N/A','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

In [57]:
print(len(data))

238


In [58]:
smodelsDF = json_normalize(data)
smodelsDF

filename  \
0      run_48_MZp_649_gB_1.154425E+00.slha   
1      run_75_MZp_851_gB_1.288135E+00.slha   
2    run_316_MZp_1000_gB_5.469112E-01.slha   
3      run_22_MZp_450_gB_1.154425E+00.slha   
..                                     ...   
234   run_215_MZp_323_gB_9.312496E-01.slha   
235   run_282_MZp_700_gB_1.154425E+00.slha   
236    run_14_MZp_352_gB_1.826514E+00.slha   
237  run_117_MZp_1151_gB_1.666373E+00.slha   

                                     SMS Decomposition  \
0    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
1    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
2    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
3    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
..                                                 ...   
234  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
235  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
236  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
237  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   

     Total xsec for missing topologies (fb)  \
0                              1.816330e+04   
1                              6.930123e+03   
2                              5.850514e+02   
3                              7.807378e+04   
..                                      ...   
234                            1.666480e+05   
235                            1.320109e+04   
236                            4.794737e+05   
237                            2.693723e+03   

     Total xsec for missing topologies with displaced decays (fb)  \
0                                         0.000000e+00              
1                                         0.000000e+00              
2                                         0.000000e+00              
3                                         0.000000e+00              
..                                                 ...              
234                                       0.000000e+00              
235                                       0.000000e+00              
236                                       0.000000e+00              
237                                       0.000000e+00              

     Total xsec for missing topologies with prompt decays (fb)  \
0                                         2.182000e+04           
1                                         8.319100e+03           
2                                         7.021899e+02           
3                                         9.422702e+04           
..                                                 ...           
234                                       2.083100e+05           
235                                       1.585400e+04           
236                                       5.906799e+05           
237                                       3.232801e+03           

     Total xsec for topologies outside the grid (fb)  OutputStatus.sigmacut  \
0                                       0.000000e+00           0.000000e+00   
1                                       0.000000e+00           0.000000e+00   
2                                       0.000000e+00           0.000000e+00   
3                                       0.000000e+00           0.000000e+00   
..                                               ...                    ...   
234                                     0.000000e+00           0.000000e+00   
235                                     0.000000e+00           0.000000e+00   
236                                     0.000000e+00           0.000000e+00   
237                                     0.000000e+00           0.000000e+00   

     OutputStatus.minmassgap  OutputStatus.maxcond  OutputStatus.ncpus  \
0               5.000000e-01          2.000000e-01                   1   
1               5.000000e-01          2.000000e-01                   1   
2               5.000000e-01          2.000000e-01                   1   
3               5.000000e-01          2.000000e-01                   1   
..      

In [59]:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 5000001])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [60]:
#Convert to DataFrame
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 8)
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [61]:
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [62]:
print('Final number of data points:',dataDF.shape[0])


Final number of data points: 238


In [63]:
dataDF.to_pickle('TRV1bb_data.pcl')
print(len(dataDF))
dataDF

238


filename  mass.5000001  width.5000001  \
0      run_48_MZp_649_gB_1.154425E+00.slha  6.491725e+02   1.141154e-02   
1      run_75_MZp_851_gB_1.288135E+00.slha  8.507492e+02   1.868931e-02   
2    run_316_MZp_1000_gB_5.469112E-01.slha  1.000000e+03   3.963475e-03   
3      run_22_MZp_450_gB_1.154425E+00.slha  4.501474e+02   7.735522e-03   
..                                     ...           ...            ...   
234   run_215_MZp_323_gB_9.312496E-01.slha  3.231884e+02   3.097745e-03   
235   run_282_MZp_700_gB_1.154425E+00.slha  7.000000e+02   1.232349e-02   
236    run_14_MZp_352_gB_1.826514E+00.slha  3.519106e+02   1.378444e-02   
237  run_117_MZp_1151_gB_1.666373E+00.slha  1.150563e+03   4.235161e-02   

        extpar.1  xsec8TeV(fb).5000001   BRs.y1.b,b   BRs.y1.c,c   BRs.y1.q,q  \
0   1.154425e+00          2.182000e+04 1.675849e-01 1.675849e-01 1.675849e-01   
1   1.288135e+00          8.319100e+03 1.669624e-01 1.669624e-01 1.669624e-01   
2   5.469112e-01          7.021900e+02 1.668188e-01 1.668188e-01 1.668188e-01   
3   1.154425e+00          9.422700e+04 1.714290e-01 1.714290e-01 1.714290e-01   
..           ...                   ...          ...          ...          ...   
234 9.312496e-01          2.083100e+05 2.000000e-01 2.000000e-01 2.000000e-01   
235 1.154425e+00          1.585400e+04 1.673337e-01 1.673337e-01 1.673337e-01   
236 1.826514e+00          5.906800e+05 1.882682e-01 1.882682e-01 1.882682e-01   
237 1.666373e+00          3.232800e+03 1.667526e-01 1.667526e-01 1.667526e-01   

     BRs.y1.t-,t+  totalxsec13TeV(fb)  totalxsec8TeV(fb)  \
0    1.620754e-01        0.000000e+00       2.182000e+04   
1    1.651880e-01        0.000000e+00       8.319100e+03   
2    1.659058e-01        0.000000e+00       7.021900e+02   
3    1.428552e-01        0.000000e+00       9.422700e+04   
..            ...                 ...                ...   
234           NaN        0.000000e+00       2.083100e+05   
235  1.633314e-01        0.000000e+00       1.585400e+04   
236  5.865891e-02        0.000000e+00       5.906800e+05   
237  1.662372e-01        0.000000e+00       3.232800e+03   

                                     SMS Decomposition  \
0    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
1    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
2    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
3    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
..                                                 ...   
234  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
235  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
236  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
237  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   

     Total xsec for missing topologies (fb)  \
0                              1.816330e+04   
1                              6.930123e+03   
2                              5.850514e+02   
3                              7.807378e+04   
..                                      ...   
234                            1.666480e+05   
235                            1.320109e+04   
236                            4.794737e+05   
237                            2.693723e+03   

     Total xsec for missing topologies with displaced decays (fb)  \
0                                         0.000000e+00              
1                                         0.000000e+00              
2                                         0.000000e+00              
3                                         0.000000e+00              
..                                                 ...              
234                                       0.000000e+00              
235                                       0.000000e+00              
236                                       0.000000e+00              
237                                       0.000000e+00              

     Total xsec for missing topologies with prompt decays (fb)  \
0                                         2.182000e+04        

In [64]:
dataDF['xsec8TeV(fb).5000001'] * dataDF['BRs.y1.b,b']

0     3.656703e+03
1     1.388977e+03
2     1.171385e+02
3     1.615324e+04
          ...     
234   4.166200e+04
235   2.652908e+03
236   1.112063e+05
237   5.390778e+02
Length: 238, dtype: float64

In [65]:
dataDF['ExptRes.result0.theory prediction (fb)']

0     3.656703e+03
1     1.388977e+03
2     1.171385e+02
3     1.615324e+04
          ...     
234   4.166200e+04
235   2.652908e+03
236   1.112063e+05
237   5.390778e+02
Name: ExptRes.result0.theory prediction (fb), Length: 238, dtype: float64

In [66]:
(dataDF['xsec8TeV(fb).5000001'] * dataDF['BRs.y1.b,b']) /dataDF['ExptRes.result0.theory prediction (fb)']

0     9.999999e-01
1     9.999999e-01
2     9.999999e-01
3     1.000000e+00
          ...     
234   1.000000e+00
235   1.000000e+00
236   9.999996e-01
237   1.000000e+00
Length: 238, dtype: float64

In [67]:
dataDF['ExptRes.result0.r']

0     1.481034e+00
1     2.365948e+00
2     2.191163e-01
3     2.261795e+00
          ...     
234   3.164627e+00
235   1.045526e+00
236   1.102034e+01
237   7.193766e-01
Name: ExptRes.result0.r, Length: 238, dtype: float64

In [68]:
(dataDF['xsec8TeV(fb).5000001'] * dataDF['BRs.y1.b,b']) /dataDF['ExptRes.result0.upper limit (fb)']

0     1.481034e+00
1     2.365948e+00
2     2.191163e-01
3     2.261795e+00
          ...     
234   3.164627e+00
235   1.045526e+00
236   1.102034e+01
237   7.193766e-01
Length: 238, dtype: float64